# #Intro To Tensorflow # 2

# by SiHyung Park / Sun-il Kim. 2019-01-19
---
- 잠시 하나만 추가하자면, 전 Step에서 만든 TensorFlow-GPU환경에 모듈을 설치하실때는 
- 꼭 터미널에서 가상환경에 들어가신 후 (source/conda activate tf-gpu(가상환경명)) pip이나 conda로 설치해주세요.


# TENSORBOARD EXPLAINED IN 5 MIN
> loss, accuracy 등 각종 지표들을 손쉽게 시각화 해봅니다

![JPEG](https://www.tensorflow.org/images/mnist_tensorboard.png)

# TENSORFLOW는 2가지 과정으로 작동됩니다
- Operations : 모델을 만들고 수많은 연산장치들을 설정해 둡니다
- Tensors : 텐서(데이터)들이 모델속에 설정된 연산장치들을 흘러서 OUTPUT까지 도달합니다

#### TENSORBOARD를 통해 작동과정 흐름을 알아볼 수 있습니다

# TENSORBOARD의 장점
- TENSOR들의 흐름을 파악할 수 있습니다
- 디버깅이 가능합니다
- 최적화 시키는데 적합합니다



In [1]:
# 이전에 했던 코드를 이용해 텐서보드를 작동시켜봅시다

import input_data
mnist = input_data.read_data_sets('./data', one_hot=True)

Extracting ./data/train-images-idx3-ubyte.gz
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz


In [4]:
import tensorflow as tf
# 학습속도를 나타내는 것으로 너무 높으면 수렴하기전에 날아가버릴 수 있고 (오버슈팅)
# 너무 낮으면 최적화 하는데 시간이 너무 오래걸릴 수 있습니다(훈련기간 동안 끝나버림)
learning_rate = 0.01
# 빨리 시각화 해보기 위해 10번만 훈련합니다
training_iteration = 10
batch_size = 100
display_step = 2

# 입력받을 데이터를 담는 '그릇'으로 MNIST 이미지가 가로28, 세로28이라
# 둘을 곱한 784를 지정해 줍니다
# 정답은 0~9 10가지이기 때문에 OUTPUT은 10으로 정해줍니다
X = tf.placeholder('float', [None, 784])
y = tf.placeholder('float', [None, 10])

# 초기에 가중치를 랜덤하게 정해줍니다. 계속 변하는 값이기에 Variable을 써줍니다
# placeholder는 변하지 않을 값(input, ouput) variable은 변하는 값을 넣어줍니다
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 텐서보드를 사용해봅니다
with tf.name_scope('Wx_b') as scope:
    # softmax를 이용한 NeuralNetwork를 사용합니다
    model = tf.nn.softmax(tf.matmul(X, W) + b)
    
with tf.name_scope('cost_function') as scope:
    # 크로스 엔트로피
    cost_function = -tf.reduce_sum(y*tf.log(model))
    # cost의 변화를 확인하기 위해 변수를 생성합니다
    tf.summary.scalar('cost_function', cost_function)
with tf.name_scope('train') as scope:
    # Gradient descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

# 나중에 가중치를 확인하기 위한 변수를 생성합니다
w_h = tf.summary.histogram('weights', W)
b_h = tf.summary.histogram('biases', b)

# 시작하기전 변수 초기화
init = tf.global_variables_initializer()

merged_summary_op = tf.summary.merge_all()

In [5]:
# 세션을 시작합니다
with tf.Session() as sess:
    sess.run(init)
    
    # summary들을 저장할 경로를 지정합니다
    summary_writer = tf.summary.FileWriter('./data/logs', graph=sess.graph)
    
    # 위에서 지정한 횟수만큼 훈련해줍니다
    for iteration in range(training_iteration):
        # cost를 0으로 초기화
        avg_cost = 0
        # mnist에서 지정한 배치사이즈(한번에 몇칸을 읽을지)로 지정
        total_batch = int(mnist.train.num_examples/batch_size)
        
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # 위에서 지정한 optimizer(GradientDescentOptimizer)를 사용
            # input인 x, 정답(label)인 y를 feed_dict에 넣어줍니다 
            sess.run(optimizer, feed_dict={X:batch_xs, y:batch_ys})
            # loss를 모두 더하고 나중에 평균으로 계산합니다
            avg_cost += sess.run(cost_function, feed_dict={X:batch_xs, y:batch_ys})/total_batch
            # 매 횟수(epoch라고도 부릅니다)마다 상태를 기록
            summary_str = sess.run(merged_summary_op, feed_dict={X: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + i)
        if iteration % display_step == 0:
            print("Iteration:", '%04d' % (iteration + 1), "cost=", "{:.9f}".format(avg_cost))
        
    print('Tuning completed!')
    
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    # 모델의 정확도를 계산하여 출력합니다. 
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({X: mnist.test.images, y: mnist.test.labels}))
 

Iteration: 0001 cost= 29.860463738
Iteration: 0003 cost= 21.066883641
Iteration: 0005 cost= 20.202680029
Iteration: 0007 cost= 19.741247816
Iteration: 0009 cost= 19.276145857
Tuning completed!
Accuracy: 0.923


# 텐서보드를 이용해 데이터 시각화를 해봅시다

#### JUPYTER NOTEBOOK HOME에서 NEW 탭을 누른 후 TERMINAL을 킵니다
- tensorboard --logdir=/home/컴퓨터사용자아이디/data/logs를 입력합니다
- /home/이후 경로는 본인이 설정한 경로로 해야합니다
- 이는 기본적으로 우분투 기반의 설명입니다.
- 윈도우의 경우에도 logs폴더의 경로를 --logdir= 뒤에 두시면 됩니다. 

#### 위 방법이 안되면 우분투 터미널을 이용해서 할 수도 있습니다
- 아나콘다 가상환경을 켠 뒤 
- (우분투의 경우 터미널에 source activate tf-gpu(가상환경명) 입력)
- (윈도우읭 경우 아나콘다 프롭트에 conda activate tf-gpu(가상환경명) 입력)
- tensorboard --logdir=/home/컴퓨터사용자아이디/data/logs
- 똑같이 입력해줍니다

#   
![JPEG](https://i.imgur.com/PLQIHm1.png)

#### 드래그 해보면서 훈련결과를 살펴봅니다
#   
#   
#   

# HOW TO MAKE NEURAL NETWORK
> 텐서플로우로 CONVOLUTION NEURAL NETWORK를 만들어봅시다

In [21]:
from __future__ import print_function
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

# MNIST 데이터를 받아옵니다
# one-hot encoding된 상태로 받아옵니다
# [house, car, tooth, car]
#
#
# [1,0,0,1], [0,1,0,0], [0,0,1,0], [0,0,0,1] 식으로 바꿔줍니다
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# hyperparameters를 설정합니다
learning_rate = 0.001
training_iters = 200000
# 한번에 128개의 sample을 입력합니다
batch_size = 128
# 훈련 10번마다 결과를 보기위해 display_step을 지정해줍니다
display_step = 10

# 28 x 28 image니까 input을 28 x 28 = 784로 지정합니다
n_input = 784
# 정답이 0~9니까 10개의 정답을 지정해줍니다
n_classes = 10
# overfitting을 피하기 위해 dropout을 지정해줍니다
# 훈련하는 동안 랜덤하게 뉴런을 탈락시켜 한쪽에만 치우쳐 훈련하는것을 방지합니다
# 모델을 generalize(regularization)해줍니다
# 뉴런의 75%만 사용하고 나머지 25%는 탈락시킵니다
dropout = 0.75

# input과 output 데이터를 담을 그릇을 지정합니다
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
# dropout
keep_prob = tf.placeholder(tf.float32)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


# STRIDE
- 몇 칸씩 옮겨다니면서 사진을 읽을지 지정해줍니다

![JPEG](https://camo.githubusercontent.com/3cb9f1b6617911669a2f8265dff7b97235d0b3dc/68747470733a2f2f63646e2d696d616765732d312e6d656469756d2e636f6d2f6d61782f313630302f312a5a436a505546724236654850526934657950366161412e676966)

# PADDING
- 원본 데이터에 필터를 적용하면 결과값은 전보다 작아집니다
- 필터 적용 후 결과 값이 작아지면 처음에 비해 특징이 많이 유실될 수 있습니다
- 이를 방지하기 위해 입력값 주위로 0을 넣어서 인위적으로 키웁니다

![JPEG](https://t1.daumcdn.net/cfile/tistory/23083C43583ED7621D)


In [22]:
# CNN의 layer를 만드는 함수를 생성합니다
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# MAXPOOLING

- M x N의 크기로 잘라낸 후 그안에서 가장 큰 값을 뽑아냅니다
- 맥스 풀링은 특징의 값이 큰 값이 다른 특징들을 대표합니다
- 전체 데이터의 사이즈가 줄어들기 때문에 컴퓨팅 리소스를 줄이고
- 데이터 크기를 줄여가며 손실이 발생하기 때문에 오버피팅을 방지할 수 있습니다
![JPEG](https://t1.daumcdn.net/cfile/tistory/2121E641583ED6AF23)

In [23]:
# maxpooling
def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')    

In [24]:
# 모델을 생성합니다
def conv_net(x, weights, biases, dropout):
    # input 사이즈를 재조정합니다
    # -1의 의미는 앞의 사이즈를 토대로 재조정할 사이즈를 추론하는것입니다
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # 첫번째 층의 convolution layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    
    # 두번째 층의 convolution layerLayer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # 모든 뉴런이 연결된 fully connected layer를 생성해줍니다
    # convolution 두개의 층을 지나치면 큰 이미지가 벡터화 됩니다
    # 이 벡터들을 가지고 fully connected layer에서 학습(연산)을 해줍니다
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    
    # output, predict 과정
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [25]:
# 가중치(weight)를 설정합니다
# 딕셔너리를 사용합니다
weights = {
    # 5x5의 conv, 1개의 input, 32개의 output
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32개의 input, 64개의 output
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    # 1024개의 input 10개의 (output)예측결과
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# 모델을 생성합니다
pred = conv_net(x, weights, biases, keep_prob)

# optimizer와 loss function을 정의합니다
# 손실함수로는 cross_entropy를 사용합니다.
# cross_entropy는 정답일 경우 0, 오답일 경우 음의 무한대로 발산합니다
# 오류가 클수록 벌을 주는 형식입니다
# softmax는 0과 1 사이의 값으로 변환해줍니다 (확률)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 모델을 평가합니다
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# 훈련하기 전, 변수들을 모두 초기화하는 변수를 만듭니다
init = tf.initialize_all_variables()

# 훈련을 시작합니다
with tf.Session() as sess:
    sess.run(init)
    step = 1
    
    # 맨처음에 정한 훈련횟수만큼 훈련합니다
    while step * batch_size < training_iters:
        # batch size를 불러옵니다
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # backpropagation을 합니다
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        # 훈련 10번마다 결과를 확인합니다
        if step % display_step == 0:
            # 오차와 정확도를 측정합니다
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # 총 결과를 출력합니다
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 19315.353516, Training Accuracy= 0.38281
Iter 2560, Minibatch Loss= 10591.512695, Training Accuracy= 0.51562
Iter 3840, Minibatch Loss= 5397.241211, Training Accuracy= 0.70312
Iter 5120, Minibatch Loss= 4887.539062, Training Accuracy= 0.76562
Iter 6400, Minibatch Loss= 1506.942871, Training Accuracy= 0.86719
Iter 7680, Minibatch Loss= 2708.647705, Training Accuracy= 0.82031
Iter 8960, Minibatch Loss= 2554.452148, Training Accuracy= 0.87500
Iter 10240, Minibatch Loss= 2522.746582, Training Accuracy= 0.85156
Iter 11520, Minibatch Loss= 1672.408936, Training Accuracy= 0.91406
Iter 12800, Minibatch Loss= 1826.131836, Training Accuracy= 0.85938
Iter 14080, Minibatch Loss= 1781.161743, Training Accuracy= 0.92188
Iter 15360, Minibatch Loss= 2800.922852, Training Accuracy= 0.87500
Iter 16640, Minibatch Loss= 1930.132080, Training Accuracy= 0.92188
Iter 17920, Minibatch Loss= 1205.521729, Training Accuracy= 0.89062
Iter 19200, Minibatch Loss= 1602.209717, Training Acc

Iter 158720, Minibatch Loss= 239.798035, Training Accuracy= 0.96094
Iter 160000, Minibatch Loss= 182.677704, Training Accuracy= 0.96094
Iter 161280, Minibatch Loss= 202.297180, Training Accuracy= 0.96094
Iter 162560, Minibatch Loss= 110.513733, Training Accuracy= 0.96875
Iter 163840, Minibatch Loss= 36.049911, Training Accuracy= 0.97656
Iter 165120, Minibatch Loss= 53.940884, Training Accuracy= 0.98438
Iter 166400, Minibatch Loss= 130.659485, Training Accuracy= 0.97656
Iter 167680, Minibatch Loss= 240.534073, Training Accuracy= 0.96094
Iter 168960, Minibatch Loss= 92.338203, Training Accuracy= 0.98438
Iter 170240, Minibatch Loss= 193.878510, Training Accuracy= 0.96094
Iter 171520, Minibatch Loss= 87.794662, Training Accuracy= 0.97656
Iter 172800, Minibatch Loss= 145.850220, Training Accuracy= 0.97656
Iter 174080, Minibatch Loss= 30.708229, Training Accuracy= 0.97656
Iter 175360, Minibatch Loss= 165.936127, Training Accuracy= 0.96875
Iter 176640, Minibatch Loss= 178.420395, Training Acc